In [1]:
# -*- coding: utf-8 -*-

"""
This program makes learning ev-gmm.
"""

'\nThis program makes learning ev-gmm.\n'

In [2]:
# __future__ module make compatible python2 and python3
from __future__ import division, print_function

# basic modules
import os
import os.path
import time

# for warning ignore
import warnings
#warning.filterwarnings('ignore')

# for file system manupulation 
from shutil import rmtree 
import glob
import argparse

# for save object
import pickle

# for make glaph
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (16, 5)
import librosa.display

# for scientific computing
import numpy as np
from numpy.linalg import norm 
from sklearn.decomposition import PCA
from sklearn.mixture import GMM # GMM class cannot use after sklearn 0.20.0
import sklearn.mixture
#from sklearn.mixture.gaussian_mixture import _compute_precision_cholesky
from sklearn.preprocessing import StandardScaler
import scipy.sparse
from scipy.signal import firwin, lfilter

# for display audio controler
from IPython.display import Audio

# for manuplate audio data
import soundfile as sf
import pyworld as pw
import pysptk
from dtw import dtw
from fastdtw import fastdtw

In [3]:
class WORLD(object):
    """
    WORLD based speech analyzer and synthezer.
    
    Ref : https://github.com/k2kobayashi/sprocket/
    """
    def __init__(self, fs=16000, fftl=1024, shiftms=5.0, minf0=40.0, maxf0=500.0):
        """
        Parameters
        ----------
        fs : int 
            Sampling frequency 
        fftl : int
            FFT length
        shiftms : float
            Shift length [ms]
        minf0 : float
            Floor in F0 estimation
        maxf0 : float
            Seli in F0 estimation
        """
        
        self.fs = fs
        self.fftl = fftl
        self.shiftms = shiftms
        self.minf0 = minf0
        self.maxf0 = maxf0
        
    def analyze(self, x):
        """
        Analyze acoustic featueres.
        
        Parameters
        ----------
        x : array, shape(`T`)
            monoral speech signal in time domain
        
        Returns
        ----------
        f0 : array, shape(`T`)
            F0 sequence
        sp : array, shape(`T`, `fftl / 2 + 1`)
            Spectral envelope sequence
        ap : array, shape(`T`, `fftl / 2 + 1`)
            aperiodicity sequence
        """
        
        f0, time_axis = pw.harvest(x, self.fs, f0_floor=self.minf0,
                                   f0_ceil=self.maxf0, frame_period=self.shiftms)
        sp = pw.cheaptrick(x, f0, time_axis, self.fs, fft_size=self.fftl)
        ap = pw.d4c(x, f0, time_axis, self.fs, fft_size=self.fftl)
        
        assert sp.shape == ap.shape
        
        return f0, sp, ap
    
    def analyze_f0(self, x):
        """
        Analyze f0.
        
        Parameters
        ----------
        x : array, shape(`T`)
            monoral speech signal in time domain
        
        Returns
        ----------
        f0 : array, shape(`T`)
            F0 sequence
        """

        f0, time_axis = pw.harvest(x, self.fs, f0_floor=self.minf0,
                                   f0_ceil=self.maxf0, frame_period=self.shiftms)
        
        assert f0.shape == x.shape()
        
        return f0
    
    def synthesis(self, f0, sp, ap):
        """
        Re-synthesizes a speech waveform from acoustic featueres.
        
        Parameters
        ----------
        f0 : array, shape(`T`)
            F0 sequence
        sp : array, shape(`T`, `fftl / 2 + 1`)
            Spectral envelope sequence
        ap : array, shape(`T`, `fftl / 2 + 1`)
            aperiodicity sequence
        """

        return pw.synthesize(f0, sp, ap, self.fs, frame_period=self.shiftms)

In [4]:
class FeatureExtractor(object):
    """
    Analyze acoustic features from a waveform.
    
    This class may have several types of estimeter like WORLD or STRAIGHT.
    Default type is WORLD.
    
    Ref : https://github.com/k2kobayashi/sprocket/
    """
    
    def __init__(self, analyzer='world', fs=16000, fftl=1024, 
                 shiftms=5.0, minf0=50.0, maxf0=500.0):
        """
        Parameters
        ----------
        analyzer : str
            Analyzer
        fs : int 
            Sampling frequency 
        fftl : int
            FFT length
        shiftms : float
            Shift length [ms]
        minf0 : float
            Floor in F0 estimation
        maxf0 : float
            Seli in F0 estimation
        """
        
        self.analyzer = analyzer
        self.fs = fs
        self.fftl = fftl
        self.shiftms = shiftms
        self.minf0 = minf0
        self.maxf0 = maxf0
    
        if self.analyzer == 'world':
            self.analyzer = WORLD(fs=self.fs, fftl=self.fftl, 
                                  minf0=self.minf0, maxf0=self.maxf0, shiftms=self.shiftms)
        else:
            raise('Analyzer Error : not support type, see FeatureExtractor class.')
        
        self._f0 = None
        self._sp = None
        self._ap = None
        
    def analyze(self, x):
        """
        Analyze acoustic featueres.
        
        Parameters
        ----------
        x : array, shape(`T`)
            monoral speech signal in time domain
        
        Returns
        ----------
        f0 : array, shape(`T`)
            F0 sequence
        sp : array, shape(`T`, `fftl / 2 + 1`)
            Spectral envelope sequence
        ap : array, shape(`T`, `fftl / 2 + 1`)
            aperiodicity sequence
        """
        
        self.x = np.array(x, dtype=np.float)
        self._f0, self._sp, self._ap = self.analyzer.analyze(self.x)
        
        # check f0 < 0
        self._f0[self._f0 < 0] = 0
        
        if np.sum(self._f0) == 0.0:
            print("Warning : F0 values are all zero.")
        
        return self._f0, self._sp, self._ap
    
    def analyze_f0(self, x):
        """
        Analyze f0.
        
        Parameters
        ----------
        x : array, shape(`T`)
            monoral speech signal in time domain
        
        Returns
        ----------
        f0 : array, shape(`T`)
            F0 sequence
        """

        self.x = np.array(x, dtype=np.float)
        self._f0 = self.analyzer.analyze_f0(self.x)

        # check f0 < 0
        self._f0[self._f0 < 0] = 0
        
        if np.sum(self._f0) == 0.0:
            print("Warning : F0 values are all zero.")
        
        return self._f0
    
    def mcep(self, dim=24, alpha=0.42):
        """
        Convert mel-cepstrum sequence from spectral envelope.
        
        Parameters
        ----------
        dim : int
            mel-cepstrum dimension
        alpha : float
            parameter of all-path filter
        
        Returns
        ----------
        mcep : array, shape(`T`, `dim + 1`)
            mel-cepstrum sequence
        """        
        
        self._analyzed_check()
        
        return pysptk.sp2mc(self._sp, dim, alpha)
    
    def codeap(self):
        """
        """
        self._analyzed_check()
        
        return pw.code_aperiodicity(self._ap, self.fs)
    
    def npow(self):
        """
        Normalized power sequence from spectral envelope.
        
        Returns
        ----------
        npow : vector, shape(`T`, `1`)
            Normalized power sequence of the given waveform
        """
        
        self._analyzed_check()
        
        npow = np.apply_along_axis(self._spvec2pow, 1, self._sp)
        
        meanpow = np.mean(npow)
        npow = 10.0 * np.log10(npow / meanpow)
        
        return npow
    
    def _spvec2pow(self, specvec):
        """
        """
        fftl2 = len(specvec) - 1
        fftl = fftl2 * 2
        
        power = specvec[0] + specvec[fftl2]
        for k in range(1, fftl2):
            power += 2.0 * specvec[k]
        power /= fftl
        
        return power
        
    def _analyzed_check(self):
        if self._f0 is None and self._sp is None and self._ap is None:
            raise('Call FeatureExtractor.analyze() before this method.')

In [5]:
class Synthesizer(object):
    """
    Synthesize a waveform from acoustic features.
    
    Ref : https://github.com/k2kobayashi/sprocket/
    """
    def __init__(self, fs=16000, fftl=1024, shiftms=5.0):
        """
        Parameters
        ----------
        fs : int 
            Sampling frequency 
        fftl : int
            FFT length
        shiftms : float
            Shift length [ms]
        """
        
        self.fs = fs
        self.fftl = fftl
        self.shiftms = shiftms
    
    def synthesis(self, f0, mcep, ap, rmcep=None, alpha=0.42):
        """
        Re-synthesizes a speech waveform from acoustic featueres.
        
        Parameters
        ----------
        f0 : array, shape(`T`)
            F0 sequence
        mcep : array, shape(`T`, `dim`)
            mel-cepstrum sequence
        ap : array, shape(`T`, `fftl / 2 + 1`)
            aperiodicity sequence
        rmcep : array, shape(`T`, `dim`)
            array of reference mel-cepstrum sequence
        alpha : float
            parameter of all-path filter
            
        Returns
        ----------
        wav : array,
            syntesized waveform
        """
        
        if rmcep is not None:
            # power modification
            mcep = mod_power(mcep, rmcep, alpha=alpha)
        
        sp = pysptk.mc2sp(mcep, alpha, self.fftl)
        wav = pw.synthesize(f0, sp, ap, self.fs, frame_period=self.shiftms)
        
        return wav
        
    def synthesis_diff(self, x, diffmcep, rmcep=None, alpha=0.42):
        """
        Re-synthesizes a speech waveform from acoustic featueres.
        filtering with a differential mel-cepstrum.
        
        Parameters
        ----------
        x : array, shape(`samples`)
            array of waveform sequence
        diffmcep : array, shape(`T`, `dim`)
            array of differential mel-cepstrum sequence
        rmcep : array, shape(`T`, `dim`)
            array of reference mel-cepstrum sequence
        alpha : float
            parameter of all-path filter
            
        Returns
        ----------
        wav : array,
            syntesized waveform
        """      
        
        x = x.astype(np.float64)
        dim = diffmcep.shape[1] - 1 
        shiftl = int(self.fs / 1000 * self.shiftms)
        
        if rmcep is not None:
            # power modification
            diffmcep = mod_power(rmcep + diffmcep, rmcep, alpha=alpha) - rmcep        
        
        # mc2b = transform mel-cepstrum to MLSA digital filter coefficients.
        b = np.apply_along_axis(pysptk.mc2b, 1, diffmcep, alpha)
        
        mlsa_fil = pysptk.synthesis.Synthesizer(pysptk.synthesis.MLSADF(dim, alpha=alpha),
                                                shiftl)
        wav = mlsa_fil.synthesis(x, b)
        
        return wav
    
    def synthesis_sp(self, f0, sp, ap):
        """
        Re-synthesizes a speech waveform from acoustic featueres.
        
        Parameters
        ----------
        f0 : array, shape(`T`)
            F0 sequence
        spc : array, shape(`T`, `dim`)
            mel-cepstrum sequence
        ap : array, shape(`T`, `fftl / 2 + 1`)
            aperiodicity sequence
            
        Returns
        ----------
        wav : array,
            syntesized waveform
        """      
        
        wav = pw.synthesize(f0, sp, ap, self.fs, frame_period=self.shiftms)
        
        return wav
    
def mod_power(cvmcep, rmcep, alpha=0.42, irlen=256):
    """
    power modification based on inpuulse responce

    Parameters
    ----------
    cvmcep : array, shape(`T`, `dim`)
        array of converted mel-cepstrum
    rmcep : arraym shape(`T`, `dim`)
        array of reference mel-cepstrum
    alpha : float
        parameter of all-path filter
    irlen : int
        Length for IIR filter

    Returns
    ----------
    modified_cvmcep : array, shape(`T`, `dim`)
        array of power modified converted mel-cepstrum
    """

    if rmcep.shape != cvmcep.shape:
        raise ValueError(
            "The shape of the converted and reference mel-cepstrum are different : {} / {}.format(cvmcep.shape, rmcep.shape)"
        )

    # mc2e = Compute energy from mel-cepstrum. e-option
    cv_e = pysptk.mc2e(cvmcep, alpha=alpha, irlen=irlen)
    r_e = pysptk.mc2e(rmcep, alpha=alpha, irlen=irlen)

    dpow = np.log(r_e / cv_e) / 2

    modified_cvmcep = np.copy(cvmcep)
    modified_cvmcep[:, 0] += dpow

    return modified_cvmcep

def low_cut_filter(x, fs, cutoff=70):
    """
    low cut filter
    
    Parameters
    ----------
    x : array, shape('samples')
        waveform sequence
    fs : array, int
        Sampling frequency
    cutoff : float
        cutoff frequency of low cut filter
    
    Returns
    ----------
    lct_x : array, shape('samples')
        Low cut filtered waveform sequence
    """
    
    nyquist = fs // 2
    norm_cutoff = cutoff / nyquist
    
    # low cut filter
    fil = firwin(255, norm_cutoff, pass_zero=False)
    lct_x = lfilter(fil, 1, x)
    
    return lct_x


In [6]:
__versions = "mcep"
__same_path = "./utterance/" + __versions + "/"
output_path = __same_path + "output/"

# 1. estimate features
feat = FeatureExtractor()
synthesizer = Synthesizer()

for files in sorted(glob.iglob("./utterance/mcep/**/**.wav", recursive=True)):
    wavf = files
    x, fs = sf.read(wavf)
    x = np.array(x, dtype=np.float)
    x = low_cut_filter(x, fs, cutoff=70)
    assert fs == 16000

    print("extract acoustic featuers: " + wavf)

    f0, sp, ap = feat.analyze(x)
    mcep = feat.mcep()

    wav = synthesizer.synthesis(f0, mcep, ap)
    wav = np.clip(wav, -32768, 32767)
    sf.write(files, wav, fs)


extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/ATR503/A/A01.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/ATR503/A/A02.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/ATR503/A/A03.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/ATR503/A/A04.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/ATR503/A/A05.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/ATR503/A/A06.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/ATR503/A/A07.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/ATR503/A/A08.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/ATR503/A/A09.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/ATR503/A/A10.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/ATR503/A/A11.wav
extract acoustic featuers: ./utterance/mcep/adaptation

extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/JSPBOBAMA1/000/A21.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/JSPBOBAMA1/000/A22.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/JSPBOBAMA1/000/A23.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/JSPBOBAMA1/000/A24.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/JSPBOBAMA1/000/A25.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/JSPBOBAMA1/000/A26.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/JSPBOBAMA1/000/A27.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/JSPBOBAMA1/000/A28.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/JSPBOBAMA1/000/A29.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/JSPBOBAMA1/000/A30.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/JSPBOBAMA1/

extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/JSUS/000/A72.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/JSUS/000/A73.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/JSUS/000/A74.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/JSUS/000/A75.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/TIMIT/000/A01.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/TIMIT/000/A02.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/TIMIT/000/A03.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/TIMIT/000/A04.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/TIMIT/000/A05.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/TIMIT/000/A06.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF01/V01/T01/TIMIT/000/A07.wav
extract acoustic featuers: ./utterance/mcep/ada

extract acoustic featuers: ./utterance/mcep/adaptation/EJF07/V01/T01/ESPBOBAMA1/000/A14.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF07/V01/T01/ESPBOBAMA1/000/A15.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF07/V01/T01/ESPBOBAMA1/000/A16.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF07/V01/T01/ESPBOBAMA1/000/A17.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF07/V01/T01/ESPBOBAMA1/000/A18.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF07/V01/T01/ESPBOBAMA1/000/A19.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF07/V01/T01/ESPBOBAMA1/000/A20.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF07/V01/T01/ESPBOBAMA1/000/A21.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF07/V01/T01/ESPBOBAMA1/000/A22.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF07/V01/T01/ESPBOBAMA1/000/A23.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF07/V01/T01/ESPBOBAMA1/

extract acoustic featuers: ./utterance/mcep/adaptation/EJF07/V01/T01/ESUS/000/A06.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF07/V01/T01/ESUS/000/A07.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF07/V01/T01/ESUS/000/A08.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF07/V01/T01/ESUS/000/A09.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF07/V01/T01/ESUS/000/A10.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF07/V01/T01/ESUS/000/A11.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF07/V01/T01/ESUS/000/A12.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF07/V01/T01/ESUS/000/A13.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF07/V01/T01/ESUS/000/A14.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF07/V01/T01/ESUS/000/A15.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJF07/V01/T01/ESUS/000/A16.wav
extract acoustic featuers: ./utterance/mcep/adaptation

...

extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/ESPBOBAMA1/000/A18.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/ESPBOBAMA1/000/A19.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/ESPBOBAMA1/000/A20.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/ESPBOBAMA1/000/A24.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/ESPBOBAMA1/000/A25.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/ESPBOBAMA1/000/A26.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/ESPBOBAMA1/000/A27.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/ESPBOBAMA1/000/A28.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/ESPBOBAMA1/000/A29.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/ESPBOBAMA1/000/A30.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/ESPBOBAMA1/

extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/ESUS/000/A19.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/ESUS/000/A20.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/ESUS/000/A21.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/ESUS/000/A23.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/ESUS/000/A24.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/ESUS/000/A26.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/ESUS/000/A27.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/ESUS/000/A28.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/ESUS/000/A29.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/ESUS/000/A30.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/ESUS/000/A31.wav
extract acoustic featuers: ./utterance/mcep/adaptation

extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/JSUS/000/A11.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/JSUS/000/A12.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/JSUS/000/A13.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/JSUS/000/A14.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/JSUS/000/A15.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/JSUS/000/A16.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/JSUS/000/A17.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/JSUS/000/A18.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/JSUS/000/A19.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/JSUS/000/A20.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/JSUS/000/A21.wav
extract acoustic featuers: ./utterance/mcep/adaptation

extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/TIMIT/000/A27.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/TIMIT/000/A28.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/TIMIT/000/A29.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/TIMIT/000/A30.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/TIMIT/000/A31.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/TIMIT/000/A32.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/TIMIT/000/A33.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/TIMIT/000/A34.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/TIMIT/000/A35.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/TIMIT/000/A36.wav
extract acoustic featuers: ./utterance/mcep/adaptation/EJM05/V01/T01/TIMIT/000/A37.wav
extract acoustic featuers: ./utterance/mcep

extract acoustic featuers: ./utterance/mcep/input/EJM10/V01/T01/TIMIT/000/A14.wav
extract acoustic featuers: ./utterance/mcep/input/EJM10/V01/T01/TIMIT/000/A17.wav
extract acoustic featuers: ./utterance/mcep/input/EJM10/V01/T01/TIMIT/000/A18.wav
extract acoustic featuers: ./utterance/mcep/input/EJM10/V01/T01/TIMIT/000/A19.wav
extract acoustic featuers: ./utterance/mcep/input/EJM10/V01/T01/TIMIT/000/A20.wav
extract acoustic featuers: ./utterance/mcep/input/EJM10/V01/T01/TIMIT/000/A21.wav
extract acoustic featuers: ./utterance/mcep/input/EJM10/V01/T01/TIMIT/000/A22.wav
extract acoustic featuers: ./utterance/mcep/input/EJM10/V01/T01/TIMIT/000/A23.wav
extract acoustic featuers: ./utterance/mcep/input/EJM10/V01/T01/TIMIT/000/A24.wav
extract acoustic featuers: ./utterance/mcep/input/EJM10/V01/T01/TIMIT/000/A25.wav
extract acoustic featuers: ./utterance/mcep/input/EJM10/V01/T01/TIMIT/000/A26.wav
extract acoustic featuers: ./utterance/mcep/input/EJM10/V01/T01/TIMIT/000/A27.wav
extract acoustic